# Using Examon with Dask

Dask is a Python library for parallel computing. https://dask.org/

This example shows how to set the examon-client to execute distributed queries and analytics with Dask.
The example covers both the JSON and the ExamonQL query interfaces.

In [1]:

import os
from examon.examon import Examon, ExamonQL
from examon.executors import ExDask


#### Configure Dask

In [2]:
from dask.distributed import Client    #v2021.8.1
client = Client(n_workers=4)  # set up local cluster 
client

<Client: 'tcp://127.0.0.1:45221' processes=4 threads=48, memory=125.66 GiB>

In [5]:
client.cluster

LocalCluster(cc5772cf, 'tcp://127.0.0.1:45221', workers=4, threads=48, memory=125.66 GiB)

#### Configure the Examon client

In [6]:

KAIROSDB_SERVER = 'examon.cineca.it'
KAIROSDB_PORT = '3000'
USER = ''
PWD = ''
ex = Examon(KAIROSDB_SERVER, port=KAIROSDB_PORT, user=USER, password=PWD, verbose=False, proxy=True)

### 1) Query in Dask using the native db interface (JSON)

In [7]:
# query some data
tstart = "12-06-2019 08:09:00"   
tstop = "19-06-2019 19:09:00"
metrics = ['Sys_Power']
tags = {'node': ['r162c16s02']}
groupby = {'name':'tag', 'tags':['node', 'plugin', 'chnl', 'cluster', 'part', 'health', 'units', 'org', 'type']}
aggrby = None

# create the dask executor, initialized with the examon-client data
ds = ExDask(ex)

# define a dask bag with the content of the query
db = ds \
    .dask(dask_client=client) \
    .query(tstart, tstop, metrics, tags, groupby=groupby, aggrby=aggrby, time_zone='Europe/Rome', batch_size=30*60*1000) \
    .to_bag(npartitions=8)

Query(tstart='12-06-2019 08:09:00', tstop='19-06-2019 19:09:00', metrics=['Sys_Power'], tags={'node': ['r162c16s02']}, groupby=[{'name': 'tag', 'tags': ['node', 'plugin', 'chnl', 'cluster', 'part', 'health', 'units', 'org', 'type']}], aggrby=None, limit=None, time_zone='Europe/Rome')


Please note: The original query is sliced in chunks over the total time range. Their size is defined by the *batch_size* parameter (milliseconds) of the *query()* method. So each Dask worker will query an amount of data defined by the number of chunks per partition. The number of partitions of the resulting bag can be defined by the *npartitions* parameter of the *to_bag()* method. If you are not sure, you can leave *npartitions=None* (default) and it will be handled by Dask as usual.

In [8]:
# Trigger the query executing an action (take) on the bag
db.flatten().take(10)

({'timestamp': 1560319740028,
  'value': 170,
  'name': 'Sys_Power',
  'node': 'r162c16s02',
  'plugin': 'confluent_pub',
  'chnl': 'data',
  'cluster': 'marconi',
  'part': 'skylake',
  'health': 'ok',
  'units': 'W',
  'org': 'cineca',
  'type': 'Current'},
 {'timestamp': 1560319800032,
  'value': 110,
  'name': 'Sys_Power',
  'node': 'r162c16s02',
  'plugin': 'confluent_pub',
  'chnl': 'data',
  'cluster': 'marconi',
  'part': 'skylake',
  'health': 'ok',
  'units': 'W',
  'org': 'cineca',
  'type': 'Current'},
 {'timestamp': 1560319860028,
  'value': 340,
  'name': 'Sys_Power',
  'node': 'r162c16s02',
  'plugin': 'confluent_pub',
  'chnl': 'data',
  'cluster': 'marconi',
  'part': 'skylake',
  'health': 'ok',
  'units': 'W',
  'org': 'cineca',
  'type': 'Current'},
 {'timestamp': 1560319920034,
  'value': 340,
  'name': 'Sys_Power',
  'node': 'r162c16s02',
  'plugin': 'confluent_pub',
  'chnl': 'data',
  'cluster': 'marconi',
  'part': 'skylake',
  'health': 'ok',
  'units': 'W',
 

In [9]:
# cretae a dask dataframe from the bag
df = db.flatten().to_dataframe()
df.head(10)

timestamp  value       name        node         plugin  chnl  cluster  \
0  1560319740028    170  Sys_Power  r162c16s02  confluent_pub  data  marconi   
1  1560319800032    110  Sys_Power  r162c16s02  confluent_pub  data  marconi   
2  1560319860028    340  Sys_Power  r162c16s02  confluent_pub  data  marconi   
3  1560319920034    340  Sys_Power  r162c16s02  confluent_pub  data  marconi   
4  1560319980026    340  Sys_Power  r162c16s02  confluent_pub  data  marconi   
5  1560320040030    350  Sys_Power  r162c16s02  confluent_pub  data  marconi   
6  1560320100027    350  Sys_Power  r162c16s02  confluent_pub  data  marconi   
7  1560320160026    340  Sys_Power  r162c16s02  confluent_pub  data  marconi   
8  1560320220006    340  Sys_Power  r162c16s02  confluent_pub  data  marconi   
9  1560320280031    320  Sys_Power  r162c16s02  confluent_pub  data  marconi   

      part health units     org     type  
0  skylake     ok     W  cineca  Current  
1  skylake     ok     W  cineca  Current  
2  skylake     ok     W  cineca  Current  
3  skylake     ok     W  cineca  Current  
4  skylake     ok     W  cineca  Current  
5  skylake     ok     W  cineca  Current  
6  skylake     ok     W  cineca  Current  
7  skylake     ok     W  cineca  Current  
8  skylake     ok     W  cineca  Current  
9  skylake     ok     W  cineca  Current

In [10]:
# show the inferred column data types of the dask dataframe
df.dtypes

timestamp     int64
value         int64
name         object
node         object
plugin       object
chnl         object
cluster      object
part         object
health       object
units        object
org          object
type         object
dtype: object

### 2) Query in Dask using the ExamonQL interface
Here we do the same query but showing how the ExamonQL interface can be used with the Dask executor

In [11]:
# create the ExamonQl object
sq = ExamonQL(ex)

# do the query as usual but get only the query data using 'get_query()'
query = sq.SELECT('*') \
    .FROM('Sys_Power') \
    .WHERE(node='r162c16s02') \
    .TSTART('12-06-2019 08:09:00') \
    .TSTOP('19-06-2019 19:09:00') \
    .get_query()

In [12]:
# pass the query data to the dask executor 
ds = ExDask(ex)

# define the dask bag passing the query parameters (query)
db = ds \
      .dask(dask_client=client) \
      .query(*query, batch_size=30*60*1000) \
      .to_bag(npartitions=8)

Query(tstart='12-06-2019 08:09:00', tstop='19-06-2019 19:09:00', metrics=['Sys_Power'], tags={'node': ['r162c16s02']}, groupby=[{'name': 'tag', 'tags': ['chnl', 'cluster', 'health', 'node', 'org', 'part', 'plugin', 'type', 'units']}], aggrby=None, limit=None, time_zone='Europe/Rome')


In [13]:
# Trigger the query executing an action (take) on the bag
db.flatten().take(10)

({'timestamp': 1560319740028,
  'value': 170,
  'name': 'Sys_Power',
  'chnl': 'data',
  'cluster': 'marconi',
  'health': 'ok',
  'node': 'r162c16s02',
  'org': 'cineca',
  'part': 'skylake',
  'plugin': 'confluent_pub',
  'type': 'Current',
  'units': 'W'},
 {'timestamp': 1560319800032,
  'value': 110,
  'name': 'Sys_Power',
  'chnl': 'data',
  'cluster': 'marconi',
  'health': 'ok',
  'node': 'r162c16s02',
  'org': 'cineca',
  'part': 'skylake',
  'plugin': 'confluent_pub',
  'type': 'Current',
  'units': 'W'},
 {'timestamp': 1560319860028,
  'value': 340,
  'name': 'Sys_Power',
  'chnl': 'data',
  'cluster': 'marconi',
  'health': 'ok',
  'node': 'r162c16s02',
  'org': 'cineca',
  'part': 'skylake',
  'plugin': 'confluent_pub',
  'type': 'Current',
  'units': 'W'},
 {'timestamp': 1560319920034,
  'value': 340,
  'name': 'Sys_Power',
  'chnl': 'data',
  'cluster': 'marconi',
  'health': 'ok',
  'node': 'r162c16s02',
  'org': 'cineca',
  'part': 'skylake',
  'plugin': 'confluent_pub'

In [14]:
# cretae a dask dataframe from the bag
df = db.flatten().to_dataframe()
df.head(10)

timestamp  value       name  chnl  cluster health        node     org  \
0  1560319740028    170  Sys_Power  data  marconi     ok  r162c16s02  cineca   
1  1560319800032    110  Sys_Power  data  marconi     ok  r162c16s02  cineca   
2  1560319860028    340  Sys_Power  data  marconi     ok  r162c16s02  cineca   
3  1560319920034    340  Sys_Power  data  marconi     ok  r162c16s02  cineca   
4  1560319980026    340  Sys_Power  data  marconi     ok  r162c16s02  cineca   
5  1560320040030    350  Sys_Power  data  marconi     ok  r162c16s02  cineca   
6  1560320100027    350  Sys_Power  data  marconi     ok  r162c16s02  cineca   
7  1560320160026    340  Sys_Power  data  marconi     ok  r162c16s02  cineca   
8  1560320220006    340  Sys_Power  data  marconi     ok  r162c16s02  cineca   
9  1560320280031    320  Sys_Power  data  marconi     ok  r162c16s02  cineca   

      part         plugin     type units  
0  skylake  confluent_pub  Current     W  
1  skylake  confluent_pub  Current     W  
2  skylake  confluent_pub  Current     W  
3  skylake  confluent_pub  Current     W  
4  skylake  confluent_pub  Current     W  
5  skylake  confluent_pub  Current     W  
6  skylake  confluent_pub  Current     W  
7  skylake  confluent_pub  Current     W  
8  skylake  confluent_pub  Current     W  
9  skylake  confluent_pub  Current     W

In [15]:
# show the inferred column data types of the dask dataframe
df.dtypes

timestamp     int64
value         int64
name         object
chnl         object
cluster      object
health       object
node         object
org          object
part         object
plugin       object
type         object
units        object
dtype: object

In [16]:
# close the dask cluster
ds.close()